# DAY 3 : Agents

## Multi Step  Tool function calling

### Supported models (https://docs.oracle.com/en-us/iaas/Content/generative-ai/chat-models.htm) 
- cohere.command-r-08-2024
- cohere.command-r-16k
- cohere.command-r-plus
- cohere.command-r-plus-08-2024


 Questions use #generative-ai-users  or #igiu-innovation-lab slack channel



In [6]:
# set up the  variables

from oci.generative_ai_inference import GenerativeAiInferenceClient
from oci.generative_ai_inference.models import OnDemandServingMode, EmbedTextDetails,CohereChatRequest, ChatDetails
import oci
import json

CONFIG_PROFILE = "AISANDBOX"
compartmentId= "ocid1.compartment.oc1..aaaaaaaaxj6fuodcmai6n6z5yyqif6a36ewfmmovn42red37ml3wxlehjmga" 
llm_service_endpoint= "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"

LLM_MODEL = "cohere.command-r-16k" 
PREAMBLE = """
        Analyze teh problem and pick teh right set of tools to answer the question
"""
MESSAGE = """
       Total sales amount over the 28th and 29th of September.
"""

## Sep up the tools

In [7]:
# report tool 
date_param = oci.generative_ai_inference.models.CohereParameterDefinition()
date_param.description = "Retrieves sales data for this day, formatted as YYYY-MM-DD."
date_param.type = "str"
date_param.is_required = True

report_tool = oci.generative_ai_inference.models.CohereTool()
report_tool.name = "query_daily_sales_report"
report_tool.description = "Connects to a database to retrieve overall sales volumes and sales information for a given day."
report_tool.parameter_definitions = {
    "date": date_param
}


In [8]:
# calculator tool 

expression_param = oci.generative_ai_inference.models.CohereParameterDefinition()
expression_param.description = "The expression to caculate."
expression_param.type = "str"
expression_param.is_required = True

calculator_tool = oci.generative_ai_inference.models.CohereTool()
calculator_tool.name = "simple_calculator"
calculator_tool.description = "Connects to a database to retrieve overall sales volumes and sales information for a given day."
calculator_tool.parameter_definitions = {
    "expression": expression_param
}

## specify the tools to use in the chat request 

In [9]:
# oci key enabled for api access
config = oci.config.from_file('~/.oci/config', CONFIG_PROFILE)

# chat request      
llm_chat_request = CohereChatRequest()
llm_chat_request.preamble_override = PREAMBLE 
llm_chat_request.message = MESSAGE
llm_chat_request.is_stream = False 
llm_chat_request.max_tokens = 500 # max token to generate, can lead to incomplete responses
llm_chat_request.is_force_single_step = False
llm_chat_request.tools = [ report_tool, calculator_tool ]



# set up chat details
chat_detail = ChatDetails()
chat_detail.serving_mode = OnDemandServingMode(model_id=LLM_MODEL)
chat_detail.compartment_id = compartmentId
chat_detail.chat_request = llm_chat_request

# set up the LLM client 
llm_client = GenerativeAiInferenceClient(
                config=config,
                service_endpoint=llm_service_endpoint,
                retry_strategy=oci.retry.NoneRetryStrategy(),
                timeout=(10,240))

## call the LLM 

In [10]:
step = 1
chat_response = llm_client.chat(chat_detail)
print(f"**************************Step {step} Result**************************")
print(f"message = {chat_response.data.chat_response.text}")
print(f"tool calls = {chat_response.data.chat_response.tool_calls}")

**************************Step 1 Result**************************
message = I will use the query_daily_sales_report tool twice, once for the 28th of September and once for the 29th, and then add the two figures together to find the total sales amount over the two days.
tool calls = [{
  "name": "query_daily_sales_report",
  "parameters": {
    "date": "2023-09-28"
  }
}]


## Call the tools 

Note: 
1. in this example we are not explicity calling the tool, we are just returning a made up response.  you will insert an explicit call to teh toolapi for real code
2. we have to keep calling chat  in a toop, so that llm can look at the tool reponse in generating it response

In [11]:
tool_results = []
llm_chat_request.message = ""
while chat_response.data.chat_response.tool_calls is not None:   # we have invoke the llm till there are no more tools left 
    for call in chat_response.data.chat_response.tool_calls: # there amay be more than one tool to call 
        tool_result = oci.generative_ai_inference.models.CohereToolResult()
        tool_result.call = call
        if call.name == "query_daily_sales_report":
            if call.parameters["date"] == "2023-09-29":
                # We should   call tool here we re simulating teh json response here 
                tool_result.outputs = [
                    {
                        "date": call.parameters["date"],
                        "summary": "Total Sales Amount: 8000, Total Units Sold: 200"
                    }
                ] 
            else:
                # We should   call tool here we re simulating teh json response here 
                tool_result.outputs = [
                    {
                        "date": call.parameters["date"],
                        "summary": "Total Sales Amount: 5000, Total Units Sold: 125"
                    }
                ] 
        else:
            # We should   call tool here we re simulating teh json response here 
            tool_result.outputs = [
                {
                    "expression": call.parameters["expression"],
                    "answer": "13000"
                }
            ]
        tool_results.append(tool_result)  # the tool responses are collectec to feed back to the llm 

    llm_chat_request.chat_history = chat_response.data.chat_response.chat_history
    llm_chat_request.tool_results = tool_results
    # call the llm again with responses from previous round of tools 
    step = step +1 
    chat_response = llm_client.chat(chat_detail)

    # Print result

    print(f"**************************Step {step} Result**************************")
    print(f"message = {chat_response.data.chat_response.text}")
    print(f"tool calls = {chat_response.data.chat_response.tool_calls}")

**************************Step 2 Result**************************
message = 
tool calls = [{
  "name": "query_daily_sales_report",
  "parameters": {
    "date": "2023-09-29"
  }
}]
**************************Step 3 Result**************************
message = The sales amount for the 28th of September is 5000 and the sales amount for the 29th of September is 8000. I will now add these two figures together.
tool calls = [{
  "name": "simple_calculator",
  "parameters": {
    "expression": "5000 + 8000"
  }
}]
**************************Step 4 Result**************************
message = The total sales amount over the 28th and 29th of September is **13,000**. The sales amount for the 28th was 5000 and the sales amount for the 29th was 8000.
tool calls = None


# Streaming version 

Steaming response reduces latency, specially if response has a lot of text. but its involved as we have to process events 

we first define the function to process the evnets 

In [12]:


def get_tool_calls_and_chat_history(chat_response):
    for event in chat_response.data.events():
        res = json.loads(event.data)
        text = res['text']
        if 'finishReason' in res:
            if 'toolCalls' in res:
                #print(f"\ntools to use : {res['toolCalls']}",flush=True)
                return text,res['toolCalls'], res['chatHistory']
            else:
                return text,None, res['chatHistory']
        else:
            if 'text' in res:
                print(res['text'], end="", flush=True)
    print("\n")
    return None, None


### call the llm in streaming mode 


In [13]:
llm_chat_request.is_stream = True
step =1 
chat_response = llm_client.chat(chat_detail)

text,tool_calls, chat_history = get_tool_calls_and_chat_history(chat_response)
print(f"\n **************************Step {step} Result**************************")
print(f"message = {text}")
print(f"tool calls = {tool_calls}")



The total sales amount over the 28th and 29th of September is **13,000**. The sales amount for the 28th of September was 5000 and the sales amount for the 29th of September was 8000.
 **************************Step 1 Result**************************
message = The total sales amount over the 28th and 29th of September is **13,000**. The sales amount for the 28th of September was 5000 and the sales amount for the 29th of September was 8000.
tool calls = None


# call tools & iterate 

In [14]:
tool_results = []
while tool_calls is not None:
    for call in tool_calls:
        call = oci.generative_ai_inference.models.CohereToolCall(**call)
        tool_result = oci.generative_ai_inference.models.CohereToolResult()
        tool_result.call = call
        if call.name == "query_daily_sales_report":
            if call.parameters["date"] == "2023-09-29":
                tool_result.outputs = [
                    {
                        "date": call.parameters["date"],
                        "summary": "Total Sales Amount: 8000, Total Units Sold: 200"
                    }
                ] 
            else:
                tool_result.outputs = [
                    {
                        "date": call.parameters["date"],
                        "summary": "Total Sales Amount: 5000, Total Units Sold: 125"
                    }
                ] 
        else:
            tool_result.outputs = [
                {
                    "expression": call.parameters["expression"],
                    "answer": "13000"
                }
            ]
        tool_results.append(tool_result)

    llm_chat_request.chat_history = chat_history
    #llm_chat_request.tool_results = tool_results
    step = step+1
    chat_response = llm_client.chat(chat_detail)

    # Print result
    print(f"\n**************************Step {step} Result**************************",flush=True)
    #print(vars(chat_response))
    text,tool_calls, chat_history = get_tool_calls_and_chat_history(chat_response)
    print(f"message = {text}")
    print(f"tool calls = {tool_calls}")


## Exercise: Clothes Recommender app 
Create an App that  Answers the Question : 
    * What clothes should I wear to Oracles headquarter tomm

1. Use following APis : https://ic-edge.ugbu.oraclepdemos.com/ash/docs
    * Weather API
    * City API
    * Clothes API
1. Things to try 
    * Conversational History/ Documents for part of information eg: gender / location etc
    * Context in preamble
